In [81]:
import re
import os
import time
import random
import warnings
import pandas as pd
import numpy as np
from typing import List
from lxml import etree as et
from bs4 import BeautifulSoup
from time import sleep
from ediblepickle import checkpoint
from urllib.parse import quote

In [53]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

opt=Options()
opt.add_argument('--no-sandbox')   
opt.add_argument('--disable-dev-shm-usage')
opt.add_argument('--disable-gpu')
warnings.filterwarnings('ignore')
#driver = webdriver.Chrome(ChromeDriverManager().install())
#driver = webdriver.Chrome(chrome_options=opt)

In [4]:
products=pd.read_csv('Skincare.csv')

#split reviews and ratings
products[['Rating','Reviews']]=products['Rating'].str.split(';',expand=True)
products['Rating']=[str(x).split(" ")[0] for x in products['Rating']]
products['Reviews']=[str(x).lstrip().split(" ")[0]for x in products['Reviews']]
products['Ingredient']=""
products['Category']=''
products['Details']=''
products['Image']=''
products.head(3)

,Brand,Name,Rating,Price,Link,ID,Reviews,Ingredient,Category,Details,Image
0,Drunk Elephant,D-Bronzi Anti-Pollution Sunshine Drops,4.6,$38.00,https://www.ulta.com/p/d-bronzi-anti-pollution...,2588470,903,,,,
1,The Ordinary,Glycolic Acid 7% Exfoliating Toning Solution,4.3,$13.00,https://www.ulta.com/p/glycolic-acid-7-exfolia...,2551155,2130,,,,
2,Estée Lauder,Advanced Night Repair Synchronized Multi-Recov...,4.5,$54.00 - $220.00,https://www.ulta.com/p/advanced-night-repair-s...,2568523,18814,,,,


In [60]:
# Fetching all Ingredients
def get_ingredients(dom):
    try:
        ingredients = dom.xpath("//*[@aria-controls='Ingredients']//p/text()")
        return re.sub(r'[\[\]\']', '', str(ingredients))
    except Exception:
        return "Ingredients are not available"

In [6]:
#Fetch product pic link
def get_pic_link(prod_soup):
    try:
        pic_list=prod_soup.find('div',class_='swiper-slide swiper-slide-active')
        return pic_list.find('img')['src']
    except Exception:
        return "Image is not available"

#https://media.ulta.com/i/ulta/2588470?w=720&fmt=webp

In [7]:
#Fetching product details
def get_detail(dom):    
    try:                                                                                
        driver.find_element("xpath",'//*[@id="Details"]').click()
        time.sleep(1)               
        details = driver.find_element("xpath", "//*[@aria-controls='Details']").text
        if details:
            return details
        else:
            return "Details are not available"
    except:                                                                              
        return "Details are not available"

In [8]:
#Fetch product category
def get_category(cat_soup):
    try:
        cat_list=cat_soup.find('ul',class_='Breadcrumbs__List')
        return cat_list.find_all('li',class_='Breadcrumbs__List--item')[2].text.strip()
    except Exception:
        return "Category is not available"

In [38]:
%%time
#Parse each product page, fetch Ingredients, Category,Detail

cache_dir = 'cache'
if not os.path.exists(cache_dir):
    os.mkdir(cache_dir)


@checkpoint(key=lambda args, kwargs: quote(str(args[1])) + '.pkl', work_dir=cache_dir)
def parse_page(df,index):
    
    prod_link=df['Link'][index]                
    driver.get(prod_link)
    #sleep(4)
    sleep(np.random.randint(2,3))
    product_soup = BeautifulSoup(driver.page_source, 'html.parser')
    dom=et.HTML(str(product_soup))
    df['Ingredient'][index]=get_ingredients(dom)
    df['Category'][index]=get_category(product_soup)
    df['Details'][index]=get_detail(dom)
    df['Image'][index]=get_pic_link(product_soup)
    if(index%200==0): 
        #back up files
        products.to_csv('ProdDetail/AllProducts{}.csv'.format(i),index=None)
    return

CPU times: user 34 µs, sys: 57 µs, total: 91 µs
Wall time: 86.1 µs


In [39]:
%%time

driver = webdriver.Chrome(chrome_options=opt)
for i in range(len(products)):
    try:
        parse_page(products,i)
    except Exception:
        driver.quit()
        driver = webdriver.Chrome(chrome_options=opt)
        parse_page(products,i)
        print(i)
        continue
driver.quit()

<timed exec>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


400


<timed exec>:6: DeprecationWarning: use options instead of chrome_options
<timed exec>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

449
600


<timed exec>:6: DeprecationWarning: use options instead of chrome_options
<timed exec>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

...

3380
3400


<timed exec>:6: DeprecationWarning: use options instead of chrome_options
<timed exec>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

3523
3600
3800
4000
4200


<timed exec>:6: DeprecationWarning: use options instead of chrome_options
<timed exec>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

4281
CPU times: user 28min 50s, sys: 1min 31s, total: 30min 22s
Wall time: 14h 39min 1s


In [55]:
products.iloc[300]

Brand                                                    CeraVe
Name                                           Eye Repair Cream
Rating                                                      4.1
Price                                                   $19.99 
Link          https://www.ulta.com/p/eye-repair-cream-xlsImp...
ID                                                      2269490
Reviews                                                    1718
Ingredient    Aqua / Water / Eau, Niacinamide, Cetyl Alcohol...
Category                                         Eye Treatments
Details       Details\nBenefits\nFormulated for delicate ski...
Image           https://media.ulta.com/i/ulta/2269490?w=78&h=78
Name: 300, dtype: object

In [56]:
products.to_csv('SkinCareDetail.csv',index=None)